In [1]:
import flywheel
import os
import datetime
from dateutil.tz import tzutc

fw = flywheel.Client()
collection_id = '5eb5081448fe1b1e5792a7a9'
local_dir = '/media/will/My Passport/Ubuntu/cortical_thickness_maps/ct'
base_date = datetime.datetime(2020, 12, 19, 20, 33, 5, 714000, tzinfo=tzutc())

In [2]:
# Get all the sessions within the collection.
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [ ]:
# Gather up the matching analyses in a list
analyses = list()
failed = list()
# Iterate over the sessions, find the analyses, and download...
for session in sessions:
    sub_label = fw.get(session.parents['subject']).label
    if '_' in sub_label:
        sub_label = sub_label.replace('_','x')
    if 'TOME' in sub_label:
        ses_label='Session'
    elif '_' in session.label:
        ses_label = session.label.replace('_','x') + '_'
    else:
        ses_label = session.label + '_'

    print(sub_label)
    print(session.label)
    print(ses_label)
    
    sub_dir = os.path.join(local_dir, sub_label)
    os.makedirs(sub_dir, exist_ok=True)
    num_files = len(os.listdir(sub_dir))
    print(num_files)
    if num_files >= 5:
        print("Already downloaded files for {}.".format(sub_label))
        print(' ')
        continue
    
    for analysis in session.analyses:
        match = False
                   
        if 'antsct' in analysis.label and analysis.created > base_date:
            match = True            

        if match and analysis.job['state']=='complete' and analysis.files:
            ## Append to the analyses list
            analyses.append(analysis)
            #file = [f for f in analysis.files if f.name.endswith('CorticalThickness.nii.gz')]
            affine_name = 'sub-{}_ses-{}TemplateToSubject1GenericAffine.mat'.format(sub_label, ses_label)
            warp_name = 'sub-{}_ses-{}TemplateToSubject0Warp.nii.gz'.format(sub_label, ses_label)
            mask_name = 'sub-{}_ses-{}CorticalMask.nii.gz'.format(sub_label, ses_label)
            ext_brain_name = 'sub-{}_ses-{}ExtractedBrain0N4.nii.gz'.format(sub_label, ses_label)
            zip_name = 'antsct_sub-{}_sub-{}.zip'.format(sub_label,sub_label)
            
            try:
                analysis.download_file_zip_member(zip_name, affine_name, os.path.join(sub_dir, affine_name))
                analysis.download_file_zip_member(zip_name, warp_name, os.path.join(sub_dir, warp_name))
                analysis.download_file_zip_member(zip_name, mask_name, os.path.join(sub_dir, mask_name))
                analysis.download_file_zip_member(zip_name, ext_brain_name, os.path.join(sub_dir, ext_brain_name))
                print("Done downloading.")
                print(' ')
            except flywheel.ApiException as e:
                print(e)
                print(session.label)
                print('')
                failed.append(analysis)
                continue

print(len(analyses))
print(len(failed))

TOMEx3001
Session 1
Session
6
Already downloaded files for TOMEx3001.
 
TOMEx3002
Session 1
Session
5
Already downloaded files for TOMEx3002.
 
TOMEx3003
Session 1
Session
5
Already downloaded files for TOMEx3003.
 
TOMEx3007
Session 1
Session
5
Already downloaded files for TOMEx3007.
 
TOMEx3004
Session 1a
Session
5
Already downloaded files for TOMEx3004.
 
TOMEx3008
Session 1
Session
5
Already downloaded files for TOMEx3008.
 
TOMEx3009
Session 1
Session
5
Already downloaded files for TOMEx3009.
 
TOMEx3005
Session 1
Session
5
Already downloaded files for TOMEx3005.
 
TOMEx3018
Session 1b
Session
5
Already downloaded files for TOMEx3018.
 
TOMEx3012
Session 1a
Session
5
Already downloaded files for TOMEx3012.
 
TOMEx3015
Session 1
Session
5
Already downloaded files for TOMEx3015.
 
TOMEx3013
Session 1
Session
5
Already downloaded files for TOMEx3013.
 
TOMEx3017
Session 1
Session
5
Already downloaded files for TOMEx3017.
 
TOMEx3011
Session 1
Session
5
Already downloaded files for TO

In [12]:
# Check how many subjects have complete downloads
complete = []
incomplete = []
subdirs = list(filter(os.path.isdir, [os.path.join(local_dir, f) for f in os.listdir(local_dir)]))
for subdir in subdirs:
    n_files = len(os.listdir(subdir))
    if n_files >= 5:
        complete.append(subdir)
    else:
        incomplete.append(subdir)
print(len(complete))
print(len(incomplete))

84
784


In [15]:
affine_name

'sub-122702_ses-122702x20180417x3T_TemplateToSubject1GenericAffine.mat'

In [16]:
zip_info = analysis.get_file_zip_info(zip_name)
print(zip_info)

{'comment': '',
 'members': [{'comment': '',
              'path': 'sub-122702_ses-122702x20180417x7T_ACTStage1Complete.txt',
              'size': 76,
              'timestamp': datetime.datetime(2020, 12, 23, 10, 12, 6, tzinfo=tzutc())},
             {'comment': '',
              'path': 'sub-122702_ses-122702x20180417x7T_ACTStage2Complete.txt',
              'size': 76,
              'timestamp': datetime.datetime(2020, 12, 23, 16, 17, 56, tzinfo=tzutc())},
             {'comment': '',
              'path': 'sub-122702_ses-122702x20180417x7T_ACTStage3Complete.txt',
              'size': 76,
              'timestamp': datetime.datetime(2020, 12, 23, 18, 17, tzinfo=tzutc())},
             {'comment': '',
              'path': 'sub-122702_ses-122702x20180417x7T_ACTStage4Complete.txt',
              'size': 76,
              'timestamp': datetime.datetime(2020, 12, 23, 23, 56, 40, tzinfo=tzutc())},
             {'comment': '',
              'path': 'sub-122702_ses-122702x20180417x7T_ACT